Force use of development local repository. The repository must be the russell_dev branch of https://github.com/russelljjarvis/neuronunit
and the docker build is: docker-stacks/neuronunit-optimization, there are more graceful ways of achieving this.

The best way will be to integrate this doc chapter5 and its supporting code 
from https://github.com/russelljjarvis/neuronunit -> https://github.com/scidash/neuronunit

On my local machine the development repo of neuronunit is located here:

```
$HOME/git
# I navigate to that directory and execute:

docker run -p 8888:8888 -v \
`pwd`:/home/jovyan/mnt scidash/neuronunit-optimization \
jupyter notebook --ip=0.0.0.0 --NotebookApp.token=\"\" \ 
--NotebookApp.disable_check_xsrf=True 
```

In [1]:
import sys
import os
THIS_DIR = os.path.dirname(os.path.realpath('chapter5.ipynb'))
this_nu = os.path.join(THIS_DIR,'../')
sys.path.insert(0,this_nu)
import neuronunit

In [ ]:
from neuronunit.tests import get_neab
from neuronunit.tests import utilities as outils                                    
from neuronunit.tests import model_parameters as modelp
import numpy as np
model = outils.model          
from sciunit import scores


Getting Rheobase cached data value for from AIBS dataset 354190013
attempting to recover from pickled file
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml
Mechanisms already loaded from path: /home/jovyan/mnt/neuronunit/tests/NeuroML2.  Aborting.
Ignoring included LEMS file: Cells.xml
Ignoring included LEMS file: Networks.xml
Ignoring included LEMS file: Simulation.xml


In [ ]:

class Score:
    def __init__(self, score):
        self.score = score

class Test:
    def _optimize(self, model,modelp):
        '''
        The implementation of optimization, consisting of implementation details.
        Inputs a model, and model parameter ranges to expore
        Private method for programmer designer.
        Outputs the optimal model, its attributes and the low error it resulted in.
        '''
        from neuronunit.tests import nsga
        gap = nsga.GAparams(model)
        # The number of generations is 3
        gap.NGEN = 3
        # The population of genes is 12
        gap.MU = 12 
        gap.BOUND_LOW = [ np.min(i) for i in modelp.model_params.values() ]
        gap.BOUND_UP = [ np.max(i) for i in modelp.model_params.values() ]
        # call the actual Genetic Algorithm with Optimization parameters: number of generation (NGEN = 3)
        # and gene population size (MU = 12)
        vmpop, pop, invalid_ind, pf = nsga.main(gap.NGEN,gap.MU,model,modelp)
        attributes = [ i.attrs for i in vmpop ]
        rheobases = [ i.rheobase for i in vmpop ]
        scores = [ i.score for i in vmpop ]        
        parameters = [ i.attrs for i in vmpop ]
        
        data_tuples = (vmpop, parameters, scores, pop, invalid_ind, pf, rheobases)
        return pop, data_tuples
        
    def _get_optimization_parameters(self, data_tuples):
        # Your specific unpacking of tuples that _optimize returns
        # vmpop, parameters, scores, pop, invalid_ind, pf, rheobases)
        _ , parameters , scores, _ , _ , _ , _ = zip(*data_tuples)
        return parameters,scores

    def optimize(self, model, modelp):
        '''
        # The Class users version of optimize
        # where details are hidden in _optimizae
        
        # Inputs: 
        # a Izhihikitch model specified in NML, but implemented with a NEURONbackend type
        # from neuronunit. Modelp a formatated dictionary of model parameters
        # where keys are Izhikitich parameters, and values are parameter ranges.
        # Outputs:
        # the optimal model, the scores as pandas data frame.
        # data_tuples: other data about models from the converged gene population
        # like resulting rheobase values from the converged genes, pandas score arrays
        # the genes corresponding to attributes of the pareto front (in a raw format).
        '''

        # Do optimization including repeated calls to judge
        models, data_tuples = self._optimize(model,modelp)
        parameters, scores = self._get_optimization_parameters(data_tuples)
        # this a way of looking at solved model parameters, ie candidate solutions from 
        # the pareto front.
        # scores is a list of pandas dataframes for the converged gene population.
        # It might be good to convert it into one big panda table if I knew how.
        return model, scores, data_tuples

    
t = Test()
model,scores,data_tuples = t.optimize(model,modelp)   

<module 'neuronunit.tests.model_parameters' from '/home/jovyan/mnt/docs/../neuronunit/tests/model_parameters.py'>
0 0.00126568483263
1 -48.4543488103
2 0.464149058258
3 31.6754763629
4 -56.4589488752
5 -61.4171345484
6 0.197605604171
7 -4.06266580506e-09
8 0.000103696734183
9 -44.9483713148
0 0.00119716708363
1 -47.9759169337
2 0.289336333127
3 39.8518426419
4 -56.816267765
5 -55.3194888841
6 0.191411615883
7 -4.38678242488e-09
8 9.80881564318e-05
9 -31.8995859241
0 0.00102942256529
1 -54.9644207902
2 0.0641482838144
3 32.5222578722
4 -55.3972904578
5 -71.417228919
6 0.122470761851
7 -4.93719079037e-09
8 9.27313062131e-05
9 -49.6160458878
0 0.00106368574622
1 -72.292790427
2 0.411825573935
3 36.2340581016
4 -56.53674238
5 -63.9644870382
6 0.101601348079
7 -4.59789284943e-09
8 9.3689390091e-05
9 -30.8933033275
0 0.000828319550637
1 -68.6179995709
2 0.875525717617
3 32.9424021017
4 -55.1951795956
5 -50.9769354503
6 0.0774593132995
7 -4.32040226907e-09
8 0.000107848639644
9 -30.1763187543

/opt/conda/lib/python3.5/site-packages/scoop-0.7.2.0-py3.5.egg/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().


3 34.9249246573
4 -56.4016738724
5 -64.9799417832
6 0.107074924744
7 -3.64439301468e-09
8 0.000106938883683
9 -39.2932190028
0 0.000749034367656
1 -70.6990595482
2 0.630063809946
3 34.3681703316
4 -59.8366755838
5 -68.3215218278
6 0.192283068333
7 -3.68332011937e-09
8 9.9996879174e-05
9 -41.4316716812
0 0.00105298837511
1 -66.6415195607
2 0.215708371869
3 32.9460621402
4 -57.7249167382
5 -73.5705303701
6 0.116983085529
7 -3.58866937762e-09
8 0.000105382452479
9 -41.6051595094
{}
{}
{}
{}
{}
{}
{}
False
{}
<class 'float'>
current 86.11111111111111 spikes 1
{}
{}
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
<class 'float'>
current 164.35185185185185 spikes 1
{}
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
<class 'float'>
current 122.68518518518519 spikes 1
{}
{}
{}
<class 'float'>
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
{}
False
{}
{}
{}
<class 'float'>
current 290.972222222

In [ ]:
modelp
import pandas as pd
models = data_tuples[1]
sc = pd.DataFrame(scores[0])
for j,i in enumerate(models):
        i.name = attributes[j]
sc

In [ ]:
data = [ models[0].name ]
model_values0 = pd.DataFrame(data)        
model_values0

In [ ]:
rheobases=data_tuples[5][0]

In [ ]:
data = [ models[0].name ]
model_values0 = pd.DataFrame(data)        
model_values0


In [ ]:
rheobases[0]


In [ ]:

sc1 = pd.DataFrame(scores[1])
sc1


In [ ]:
rheobases[1]


In [ ]:
data=[ models[1].name ]
model_values1 = pd.DataFrame(data)        
model_values1

In [ ]:
models[1].name        

The code below is used to get the differences between values obtained via brute force, and those obtained otherwise.
It displays the differences in parameter values as pandas data tables.
    
I have knowingly violated Github conventions by adding data (a pickled file, as well as sources to the repository). 
The justification being that ground_error (the ground truth to compare against Genetic Algorithm outputs). 
Takes a prohibitively long time to generate, and therefore detracts from notebooking philosophy.

In [ ]:
import pickle
import pandas as pd

try:
    ground_error = pickle.load(open('big_model_evaulated.pickle','rb'))
except:
    # The exception code is only skeletal, it would not actually work, but its the right principles.
    print('{0} it seems the error truth data does not yet exist, lets create it now '.format(str(False)))
    ground_error = list(futures.map(outils.func2map, ground_truth))
    pickle.dump(ground_error,open('big_model_evaulated.pickle','wb'))

# ground_error_nsga=list(zip(vmpop,pop,invalid_ind))
# pickle.dump(ground_error_nsga,open('nsga_evaulated.pickle','wb'))

sum_errors = [ i[0] for i in ground_error ]
composite_errors = [ i[1] for i in ground_error ]
attrs = [ i[2] for i in ground_error ]
rheobase = [ i[3] for i in ground_error ]

indexs = [i for i,j in enumerate(sum_errors) if j==np.min(sum_errors) ][0]
indexc = [i for i,j in enumerate(composite_errors) if j==np.min(composite_errors) ][0]
#assert indexs == indexc
vmpop = data_tuples[0]
df_0 = pd.DataFrame([ (k,v,vmpop[0].attrs[k],float(v)-float(vmpop[0].attrs[k])) for k,v in ground_error[indexc][2].items() ])
df_1 = pd.DataFrame([ (k,v,vmpop[1].attrs[k],float(v)-float(vmpop[1].attrs[k])) for k,v in ground_error[indexc][2].items() ])




In [ ]:
#These are the differences in attributes found via brute force versus the genetic algorithm. For the top two candidates.

df_0

In [ ]:
df_1
